In [ ]:
import re
from random import randint
from datetime import datetime

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt


In [ ]:
data=pd.read_csv(r"/data/Reporte_Hurto_por_Modalidades_Polic_a_Nacional.csv").drop_duplicates()


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
data

,DEPARTAMENTO,MUNICIPIO,CODIGO DANE,ARMAS MEDIOS,FECHA HECHO,GENERO,GRUPO ETARIO,TIPO DE HURTO,CANTIDAD,fecha,año,mes
0,CAQUETÁ,SAN VICENTE DEL CAGUÁN,18753000,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,FEMENINO,ADULTOS,HURTO RESIDENCIAS,1,2010-01-01,2010,1
1,CAUCA,POPAYÁN (CT),19001000,ARMA BLANCA / CORTOPUNZANTE,1/01/2010,FEMENINO,ADULTOS,HURTO RESIDENCIAS,2,2010-01-01,2010,1
2,HUILA,PITALITO,41551000,ARMA DE FUEGO,1/01/2010,MASCULINO,ADULTOS,HURTO RESIDENCIAS,1,2010-01-01,2010,1
3,VALLE,JAMUNDÍ,76364000,ARMA DE FUEGO,1/01/2010,MASCULINO,ADULTOS,HURTO RESIDENCIAS,1,2010-01-01,2010,1
4,AMAZONAS,LETICIA (CT),91001000,CONTUNDENTES,1/01/2010,MASCULINO,ADULTOS,HURTO RESIDENCIAS,1,2010-01-01,2010,1
...,...,...,...,...,...,...,...,...,...,...,...,...
519383,MAGDALENA,SANTA MARTA (CT),47001000,NO REPORTADO,28/02/2022,FEMENINO,ADULTOS,HURTO RESIDENCIAS,1,2022-02-28,2022,2
519384,SANTANDER,BUCARAMANGA (CT),68001000,NO REPORTADO,30/03/2022,FEMENINO,ADULTOS,HURTO RESIDENCIAS,1,2022-03-30,2022,3
519385,SUCRE,MAJAGUAL,70429000,NO REPORTADO,12/01/2022,MASCULINO,ADULTOS,HURTO RESIDENCIAS,1,2022-12-01,2022,12
519386,SUCRE,SAN LUIS DE SINCÉ,70742000,NO REPORTADO,21/02/2022,MASCULINO,ADULTOS,HURTO RESIDENCIAS,1,2022-02-21,2022,2


In [ ]:
for i in data.columns:
  print(data[data[i].isnull()])

Empty DataFrame
Columns: [DEPARTAMENTO, MUNICIPIO, CODIGO DANE, ARMAS MEDIOS, FECHA HECHO, GENERO, GRUPO ETARIO, TIPO DE HURTO, CANTIDAD]
Index: []
Empty DataFrame
Columns: [DEPARTAMENTO, MUNICIPIO, CODIGO DANE, ARMAS MEDIOS, FECHA HECHO, GENERO, GRUPO ETARIO, TIPO DE HURTO, CANTIDAD]
Index: []
Empty DataFrame
Columns: [DEPARTAMENTO, MUNICIPIO, CODIGO DANE, ARMAS MEDIOS, FECHA HECHO, GENERO, GRUPO ETARIO, TIPO DE HURTO, CANTIDAD]
Index: []
Empty DataFrame
Columns: [DEPARTAMENTO, MUNICIPIO, CODIGO DANE, ARMAS MEDIOS, FECHA HECHO, GENERO, GRUPO ETARIO, TIPO DE HURTO, CANTIDAD]
Index: []
Empty DataFrame
Columns: [DEPARTAMENTO, MUNICIPIO, CODIGO DANE, ARMAS MEDIOS, FECHA HECHO, GENERO, GRUPO ETARIO, TIPO DE HURTO, CANTIDAD]
Index: []
Empty DataFrame
Columns: [DEPARTAMENTO, MUNICIPIO, CODIGO DANE, ARMAS MEDIOS, FECHA HECHO, GENERO, GRUPO ETARIO, TIPO DE HURTO, CANTIDAD]
Index: []
        DEPARTAMENTO         MUNICIPIO CODIGO DANE         ARMAS MEDIOS  \
229281  CUNDINAMARCA  BOGOTÁ D.C. (CT

Existen datos Nan en el grupo etario, de resto el conjunto de datos parece completos.

In [ ]:
print(data["DEPARTAMENTO"].unique())
print(data["MUNICIPIO"].unique())

['CAQUETÁ' 'CAUCA' 'HUILA' 'VALLE' 'AMAZONAS' 'BOYACÁ' 'CASANARE'
 'CUNDINAMARCA' 'MAGDALENA' 'META' 'NORTE DE SANTANDER' 'SANTANDER'
 'SUCRE' 'TOLIMA' 'ANTIOQUIA' 'CHOCÓ' 'SAN ANDRÉS' 'VAUPÉS' 'ARAUCA'
 'BOLÍVAR' 'CESAR' 'CÓRDOBA' 'PUTUMAYO' 'NARIÑO' 'ATLÁNTICO' 'CALDAS'
 'RISARALDA' 'GUAJIRA' 'QUINDÍO' 'VICHADA' 'GUAINÍA' 'GUAVIARE'
 'NO REPORTADO']
['SAN VICENTE DEL CAGUÁN' 'POPAYÁN (CT)' 'PITALITO' ... 'ALTO BAUDÓ'
 'NO REPORTADO' 'EL CANTÓN DEL SAN PABLO']


Los datos de departamento y municipios parecen estar bien denominados


2.1

In [ ]:
data.groupby("DEPARTAMENTO").sum().sort_values(["CANTIDAD"])[-3:]

,CANTIDAD
DEPARTAMENTO,
VALLE,60497
ANTIOQUIA,78134
CUNDINAMARCA,206705


Los Departamentos más afectados fueron Valle, Antioquia y Cundinamarca

2.2

In [ ]:
data.groupby("ARMAS MEDIOS").sum().sort_values(["CANTIDAD"])[-5:]

,CANTIDAD
ARMAS MEDIOS,
ARMA BLANCA / CORTOPUNZANTE,14475
PALANCAS,54920
ARMA DE FUEGO,75239
CONTUNDENTES,161606
SIN EMPLEO DE ARMAS,372401


Principalmente se han realizado hurtos sin armas, posteriormente contundentes, armas de fuego, palancas y cortopunzantes

2.3

In [ ]:
data["fecha"]=pd.to_datetime(data["FECHA HECHO"])
data["año"]=data["fecha"].dt.year

In [ ]:
summge=data.groupby(["año","GENERO"]).sum().reset_index()
summet=data.groupby(["año","GRUPO ETARIO"]).sum().reset_index()

In [ ]:
summge

,año,GENERO,CANTIDAD
0,2010,FEMENINO,8086
1,2010,MASCULINO,9344
2,2010,NO REPORTA,9
3,2010,NO REPORTADO,12912
4,2011,FEMENINO,8269
5,2011,MASCULINO,9692
6,2011,NO REPORTA,7
7,2011,NO REPORTADO,15283
8,2012,FEMENINO,10202
9,2012,MASCULINO,12142


Al principio eran un poco mayores los crimenes relacionados con personas  masculinas que femeninas. Esto permanece en el tiempo, pero aumentan la cantidad de crimenes en ambos casos

In [ ]:
2.4

In [ ]:
data["mes"]=data["fecha"].dt.month
print(data.groupby("mes").sum())

     CANTIDAD       año
mes                    
1       69101  97170098
2       59817  83298924
3       62018  86059493
4       58648  81374528
5       60199  83507773
6       57010  79932485
7       58269  82272134
8       58945  82408764
9       57063  80339513
10      59159  83060998
11      55669  78176063
12      54500  76451467


El primer mes del año tiene muchos mas casos que los demás por lo que es posible que haya alguna tendencia fuerte para enero, o la gente este mas deseperada por robar


In [ ]:
2.5

In [ ]:
print(data.groupby("TIPO DE HURTO").sum())

                             CANTIDAD        año      mes
TIPO DE HURTO                                            
HURTO ENTIDADES COMERCIALES    360194  438813237  1375831
HURTO RESIDENCIAS              350204  555239003  1753848


Solo hay dos, y en ambos casos son bastantes similares la cantidad de hurtos que hay tanto a entidades comerciales como a residencias